Import libraries/functions

In [33]:
import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt

Load the data

In [34]:
hpi_raw = pd.read_csv(filepath_or_buffer="../data/UK-HPI-full-file-2025-05.csv")
hpi_raw

,Date,RegionName,AreaCode,AveragePrice,Index,IndexSA,1m%Change,12m%Change,AveragePriceSA,SalesVolume,...,NewPrice,NewIndex,New1m%Change,New12m%Change,NewSalesVolume,OldPrice,OldIndex,Old1m%Change,Old12m%Change,OldSalesVolume
0,01/01/2004,Aberdeenshire,S12000034,84638,41.1,NaN,NaN,NaN,NaN,388.0,...,112843.0,40.7,NaN,NaN,103.0,81273.0,41.0,NaN,NaN,285.0
1,01/02/2004,Aberdeenshire,S12000034,84623,41.1,NaN,0.0,NaN,NaN,326.0,...,113061.0,40.8,0.2,NaN,107.0,81194.0,40.9,-0.1,NaN,219.0
2,01/03/2004,Aberdeenshire,S12000034,86536,42.1,NaN,2.3,NaN,NaN,453.0,...,115218.0,41.6,1.9,NaN,140.0,83137.0,41.9,2.4,NaN,313.0
3,01/04/2004,Aberdeenshire,S12000034,87373,42.5,NaN,1.0,NaN,NaN,571.0,...,115247.0,41.6,0.0,NaN,180.0,84241.0,42.5,1.3,NaN,391.0
4,01/05/2004,Aberdeenshire,S12000034,89493,43.5,NaN,2.4,NaN,NaN,502.0,...,117377.0,42.4,1.8,NaN,167.0,86466.0,43.6,2.6,NaN,335.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146245,01/01/2025,Yorkshire and The Humber,E12000003,201658,105.2,105.4,-0.5,5.2,201954.0,3950.0,...,305006.0,112.9,1.7,10.4,36.0,197965.0,104.9,-0.6,5.0,3914.0
146246,01/02/2025,Yorkshire and The Humber,E12000003,206561,107.8,107.8,2.4,8.1,206572.0,4234.0,...,358799.0,132.9,17.6,30.2,30.0,200504.0,106.2,1.3,6.6,4204.0
146247,01/03/2025,Yorkshire and The Humber,E12000003,212483,110.9,111.1,2.9,10.2,212922.0,6591.0,...,371625.0,137.6,3.6,33.4,42.0,206138.0,109.2,2.8,8.6,6549.0
146248,01/04/2025,Yorkshire and The Humber,E12000003,198997,103.8,104.6,-6.3,3.7,200437.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
### Filter for England Wales (area code in hpi index identifies England and Wales by starting the area code with E or W. K stands for UK)
hpi_raw_ew = hpi_raw[hpi_raw["AreaCode"].str.startswith(("E", "W"))]

### Check that only england and wales are here
hpi_raw_ew.AreaCode.str[0].unique()

array(['E', 'W'], dtype=object)

In [36]:
hpi_raw_ew['Date'] = pd.to_datetime(hpi_raw_ew['Date'])

/tmp/ipykernel_3783/3522449281.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hpi_raw_ew['Date'] = pd.to_datetime(hpi_raw_ew['Date'])


In [37]:
hpi_raw_ew

,Date,RegionName,AreaCode,AveragePrice,Index,IndexSA,1m%Change,12m%Change,AveragePriceSA,SalesVolume,...,NewPrice,NewIndex,New1m%Change,New12m%Change,NewSalesVolume,OldPrice,OldIndex,Old1m%Change,Old12m%Change,OldSalesVolume
257,1995-01-01,Adur,E07000223,54669,14.5,NaN,NaN,NaN,NaN,79.0,...,51937.0,15.9,NaN,NaN,11.0,54271.0,14.4,NaN,NaN,68.0
258,1995-01-02,Adur,E07000223,55864,14.8,NaN,2.2,NaN,NaN,69.0,...,53163.0,16.2,2.4,NaN,5.0,55454.0,14.7,2.2,NaN,64.0
259,1995-01-03,Adur,E07000223,55880,14.8,NaN,0.0,NaN,NaN,103.0,...,53321.0,16.3,0.3,NaN,5.0,55464.0,14.7,0.0,NaN,98.0
260,1995-01-04,Adur,E07000223,55596,14.7,NaN,-0.5,NaN,NaN,84.0,...,53003.0,16.2,-0.6,NaN,2.0,55184.0,14.6,-0.5,NaN,82.0
261,1995-01-05,Adur,E07000223,53483,14.2,NaN,-3.8,NaN,NaN,77.0,...,51018.0,15.6,-3.7,NaN,5.0,53086.0,14.1,-3.8,NaN,72.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146245,2025-01-01,Yorkshire and The Humber,E12000003,201658,105.2,105.4,-0.5,5.2,201954.0,3950.0,...,305006.0,112.9,1.7,10.4,36.0,197965.0,104.9,-0.6,5.0,3914.0
146246,2025-01-02,Yorkshire and The Humber,E12000003,206561,107.8,107.8,2.4,8.1,206572.0,4234.0,...,358799.0,132.9,17.6,30.2,30.0,200504.0,106.2,1.3,6.6,4204.0
146247,2025-01-03,Yorkshire and The Humber,E12000003,212483,110.9,111.1,2.9,10.2,212922.0,6591.0,...,371625.0,137.6,3.6,33.4,42.0,206138.0,109.2,2.8,8.6,6549.0
146248,2025-01-04,Yorkshire and The Humber,E12000003,198997,103.8,104.6,-6.3,3.7,200437.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
hpi_raw_ew_ts = []

# Monthly frequency
freq = 'MS'

for area in hpi_raw_ew['AreaCode'].unique():
    area_df = hpi_raw_ew[hpi_raw_ew['AreaCode'] == area].copy()
    
    # Set date index and ensure proper frequency
    area_df = area_df.set_index('Date').sort_index().asfreq(freq)
    
    # Optionally interpolate missing values
    area_df['AveragePrice'] = area_df['AveragePrice'].interpolate()
    
    # Perform decomposition
    decomposition = seasonal_decompose(area_df['AveragePrice'], model='additive', period=12)
    
    # Add components back to the DataFrame
    area_df['trend'] = decomposition.trend
    area_df['seasonal'] = decomposition.seasonal
    area_df['residual'] = decomposition.resid
    area_df['area_code'] = area  # Re-add if needed

    # Reset index so date is a column again
    area_df = area_df.reset_index()
    
    # Store for concatenation
    hpi_raw_ew_ts.append(area_df)

# Combine all area_code DataFrames into one
final_df = pd.concat(hpi_raw_ew_ts, ignore_index=True)

ValueError: This function does not handle missing values